## Common

In [ ]:
# Opts
import os
import argparse

# Utils
import numpy as np
import random

# Train Generator
import sys
from tensorflow import keras

# Trainer
import glob
import math
import time

# ESC-NAS
import datetime
import subprocess
import gc
import re
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

# import logging
# tf.get_logger().setLevel(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

cd (Change directory) to the location of the folders


In [ ]:
%cd /content/drive/MyDrive/ESCNAS

In [ ]:
def parse():
    parser = argparse.ArgumentParser(description='ESC-NAS Sound Classification')

    # General settings
    parser.add_argument('--netType', default='ESC-NAS',  required=False)
    parser.add_argument('--data', default='{}/datasets/'.format(os.getcwd()),  required=False)
    parser.add_argument('--dataset', required=False, default='urbansound8k', choices=['fsc22', 'esc10', 'esc50', 'urbansound8k'])
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning')
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation')

    opt = parser.parse_args(args=[])

    # Learning settings
    opt.batchSize = 128
    opt.weightDecay = 5e-4
    opt.momentum = 0.9
    opt.nEpochs = 150
    opt.LR = 0.01
    opt.schedule = [0.2, 0.4, 0.7]  # [0-10 = 10-3, 11-30 = 10-2, 31-60 = 10-3, 61-105=10-4, 106-150=10-5]
    opt.warmup = 10

    # Basic Net Settings
    opt.nClasses = 10
    opt.nFolds = 5
    opt.splits = [i for i in range(1, opt.nFolds + 1)]
    opt.sr = 20000
    opt.inputLength = 30225
    opt.mixupFactor = 2

    # Test data
    opt.nCrops = 10

    opt.augmentation_data = {"time_stretch": 0.8, "pitch_shift": 1.5}

    opt.class_labels = ["air_conditioner", "car_horn", "children_playing", "dog_bark", "drilling",
                        "engine_idling", "gun_shot", "jackhammer", "siren", "street_music"]

    return opt

def display_info(opt):
    print('+------------------------------+')
    print('| {} Sound classification'.format(opt.netType))
    print('+------------------------------+')
    print('| dataset  : {}'.format(opt.dataset))
    print('| nEpochs  : {}'.format(opt.nEpochs))
    print('| LRInit   : {}'.format(opt.LR))
    print('| schedule : {}'.format(opt.schedule))
    print('| warmup   : {}'.format(opt.warmup))
    print('| batchSize: {}'.format(opt.batchSize))
    print('| Splits: {}'.format(opt.splits))
    print('+------------------------------+')

### Utils

In [ ]:
# Fixed seed for reproducability
random.seed(42)
# Default data augmentation
def padding(pad):
    def f(sound):
        return np.pad(sound, pad, 'constant')

    return f

def random_crop(size):
    def f(sound):
        org_size = len(sound)
        start = random.randint(0, org_size - size)
        return sound[start: start + size]

    return f

def normalize(factor):
    def f(sound):
        return sound / factor

    return f

# For strong data augmentation
def random_scale(max_scale, interpolate='Linear'):
    def f(sound):
        scale = np.power(max_scale, random.uniform(-1, 1))
        output_size = int(len(sound) * scale)
        ref = np.arange(output_size) / scale
        if interpolate == 'Linear':
            ref1 = ref.astype(np.int32)
            ref2 = np.minimum(ref1 + 1, len(sound) - 1)
            r = ref - ref1
            scaled_sound = sound[ref1] * (1 - r) + sound[ref2] * r
        elif interpolate == 'Nearest':
            scaled_sound = sound[ref.astype(np.int32)]
        else:
            raise Exception('Invalid interpolation mode {}'.format(interpolate))

        return scaled_sound

    return f

def random_gain(db):
    def f(sound):
        return sound * np.power(10, random.uniform(-db, db) / 20.0)

    return f

# For testing phase
def multi_crop(input_length, n_crops):
    def f(sound):
        stride = (len(sound) - input_length) // (n_crops - 1)
        sounds = [sound[stride * i: stride * i + input_length] for i in range(n_crops)]
        return np.array(sounds)

    return f

# For BC learning
def a_weight(fs, n_fft, min_db=-80.0):
    freq = np.linspace(0, fs // 2, n_fft // 2 + 1)
    freq_sq = np.power(freq, 2)
    freq_sq[0] = 1.0
    weight = 2.0 + 20.0 * (2 * np.log10(12194) + 2 * np.log10(freq_sq)
                           - np.log10(freq_sq + 12194 ** 2)
                           - np.log10(freq_sq + 20.6 ** 2)
                           - 0.5 * np.log10(freq_sq + 107.7 ** 2)
                           - 0.5 * np.log10(freq_sq + 737.9 ** 2))
    weight = np.maximum(weight, min_db)

    return weight

def compute_gain(sound, fs, min_db=-80.0, mode='A_weighting'):
    if fs == 16000 or fs == 20000:
        n_fft = 2048
    elif fs == 44100:
        n_fft = 4096
    else:
        raise Exception('Invalid fs {}'.format(fs))
    stride = n_fft // 2

    gain = []

    for i in range(0, len(sound) - n_fft + 1, stride):
        if mode == 'RMSE':
            g = np.mean(sound[i: i + n_fft] ** 2)
        elif mode == 'A_weighting':
            spec = np.fft.rfft(np.hanning(n_fft + 1)[:-1] * sound[i: i + n_fft])
            power_spec = np.abs(spec) ** 2
            a_weighted_spec = power_spec * np.power(10, a_weight(fs, n_fft) / 10)
            g = np.sum(a_weighted_spec)
        else:
            raise Exception('Invalid mode {}'.format(mode))
        gain.append(g)

    gain = np.array(gain)
    gain = np.maximum(gain, np.power(10, min_db / 10))
    gain_db = 10 * np.log10(gain)

    return gain_db

def mix(sound1, sound2, r, fs):
    gain1 = np.max(compute_gain(sound1, fs))  # Decibel
    gain2 = np.max(compute_gain(sound2, fs))
    t = 1.0 / (1 + np.power(10, (gain1 - gain2) / 20.) * (1 - r) / r)
    sound = ((sound1 * t + sound2 * (1 - t)) / np.sqrt(t ** 2 + (1 - t) ** 2))

    return sound

# Convert time representation
def to_hms(time):
    h = int(time // 3600)
    m = int((time - h * 3600) // 60)
    s = int(time - h * 3600 - m * 60)
    if h > 0:
        line = '{}h{:02d}m'.format(h, m)
    else:
        line = '{}m{:02d}s'.format(m, s)

    return line

## Resources

### Train Generator

In [ ]:
class Generator(keras.utils.Sequence):
    # Generates data for Keras
    def __init__(self, samples, labels, options):
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))]
        self.opt = options
        self.batch_size = options.batchSize
        self.preprocess_funcs = self.preprocess_setup()

    def __len__(self):
        # Denotes the number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, batchIndex):
        # Generate one batch of data
        batchX, batchY = self.generate_batch(batchIndex)
        batchX = np.expand_dims(batchX, axis=1)
        batchX = np.expand_dims(batchX, axis=3)
        return batchX, batchY

    def generate_batch(self, batchIndex):
        # Generates data containing batch_size samples
        sounds = []
        labels = []
        selected = []
        indexes = None
        
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                ind1 = random.randint(0, len(self.data) - 1)
                ind2 = random.randint(0, len(self.data) - 1)

                sound1, label1 = self.data[ind1]
                sound2, label2 = self.data[ind2]

                if len({label1, label2}) == 2 and "{}-{}".format(ind1, ind2) not in selected:
                    selected.append("{}-{}".format(ind1, ind2))
                    break
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random())
            sound = mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            eye = np.eye(self.opt.nClasses)
            label = (eye[label1 - 1] * r + eye[label2 - 1] * (1 - r)).astype(np.float32)

            # For stronger augmentation
            sound = random_gain(6)(sound).astype(np.float32)

            sounds.append(sound)
            labels.append(label)

        sounds = np.asarray(sounds)
        labels = np.asarray(labels)

        return sounds, labels

    def preprocess_setup(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [random_scale(1.25)]

        funcs += [padding(self.opt.inputLength // 2),
                  random_crop(self.opt.inputLength),
                  normalize(32768.0)]
        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound

def setup(opt, split):
    dataset = np.load(os.path.join(opt.data, opt.dataset, 'wav{}.npz'.format(opt.sr // 1000)), allow_pickle=True)
    train_sounds = []
    train_labels = []
    
    for i in range(1, opt.nFolds + 1):
        sounds = dataset['fold{}'.format(i)].item()['sounds']
        labels = dataset['fold{}'.format(i)].item()['labels']
        if i != split:
            train_sounds.extend(sounds)
            train_labels.extend(labels)

    trainGen = Generator(train_sounds, train_labels, opt)

    return trainGen

## TF

### Trainer

In [ ]:
class Trainer:
    def __init__(self, opt=None):
        self.opt = opt
        self.trainGen = setup(self.opt, self.opt.split)

    def GetLR(self, epoch):
        divide_epoch = np.array([self.opt.nEpochs * i for i in self.opt.schedule])
        decay = sum(epoch > divide_epoch)
        if epoch <= self.opt.warmup:
            decay = 1
        return self.opt.LR * np.power(0.1, decay)

class CustomCallback(keras.callbacks.Callback):
    def __init__(self, opt):
        self.opt = opt
        self.testX = None
        self.testY = None
        self.curEpoch = 0
        self.curLr = opt.LR
        self.cur_epoch_start_time = time.time()
        self.bestAcc = 0.0
        self.bestAccEpoch = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.curEpoch = epoch+1
        self.curLr = Trainer(self.opt).GetLR(epoch+1)
        self.cur_epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        train_time = time.time() - self.cur_epoch_start_time
        self.load_test_data()
        val_acc, val_loss = self.validate(self.model)
        logs['val_acc'] = val_acc
        logs['val_loss'] = val_loss
        if val_acc > self.bestAcc:
            self.bestAcc = val_acc
            self.bestAccEpoch = epoch + 1
        epoch_time = time.time() - self.cur_epoch_start_time
        val_time = epoch_time - train_time
        line = 'SP-{}, Epoch: {}/{} | Time: {} (Train {}  Val {}) | Train: LR {}  Loss {:.2f}  Acc {:.2f}% | Val: Loss {:.2f}  Acc(top1) {:.2f}% | HA {:.2f}@{}\n'.format(
            self.opt.split, epoch+1, self.opt.nEpochs, to_hms(epoch_time), to_hms(train_time), to_hms(val_time),
            self.curLr, logs['loss'], logs['accuracy']*100 if 'accuracy' in logs else logs['acc']*100, val_loss, val_acc, self.bestAcc, self.bestAccEpoch)
        sys.stdout.write(line)
        sys.stdout.flush()

    def load_test_data(self):
        if self.testX is None:
            data = np.load(os.path.join(self.opt.data, self.opt.dataset,
                                        'test_data_{}khz/fold{}_test800.npz'.format(self.opt.sr // 1000,
                                                                                     self.opt.split)),
                           allow_pickle=True)

            self.testX = data['x']
            self.testY = data['y']

    def validate(self, model):
        y_pred = None
        y_target = None
        batch_size = (self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops
        
        for batchIndex in range(math.ceil(len(self.testX) / batch_size)):
            x = self.testX[batchIndex*batch_size : (batchIndex+1)*batch_size]
            y = self.testY[batchIndex*batch_size : (batchIndex+1)*batch_size]
            scores = model.predict(x, batch_size=len(y), verbose=0)
            y_pred = scores if y_pred is None else np.concatenate((y_pred, scores))
            y_target = y if y_target is None else np.concatenate((y_target, y))

        acc, loss = self.compute_accuracy(y_pred, y_target)
        return acc, loss

    # Calculating average prediction (10 crops) and final accuracy
    def compute_accuracy(self, y_pred, y_target):
        # Reshape y_pred to shape it like each sample comtains 10 samples.
        if self.opt.nCrops > 1:
            y_pred = (y_pred.reshape(y_pred.shape[0]//self.opt.nCrops, self.opt.nCrops, y_pred.shape[1])).mean(axis=1)
            y_target = (y_target.reshape(y_target.shape[0]//self.opt.nCrops, self.opt.nCrops, y_target.shape[1])).mean(axis=1)

        loss = keras.losses.KLD(y_target, y_pred).numpy().mean()

        #Get the indices that has highest average value for each sample
        y_pred = y_pred.argmax(axis=1) + 1
        y_target = y_target.argmax(axis=1) + 1
        accuracy = (y_pred==y_target).mean()*100

        return accuracy, loss

### Model

In [ ]:
class ESCNAS_mixup :
    architecture_name = 'resulting_architecture'
    def __init__(self, max_RAM, max_Flash, val_split, input_shape, save_path='./', opt=None, n_class=10) :
        self.max_Flash = max_Flash
        self.max_RAM = max_RAM
        self.num_classes = n_class
        self.val_split = val_split
        self.input_shape = input_shape
        self.save_path = save_path

        self.path_to_trained_models = f"{self.save_path}/trained_models"
        os.makedirs(self.path_to_trained_models)

        display_info(opt)
        self.trainer = Trainer(opt)

    # k base number of kernels of the convolutional layers
    # c number of cells added upon the first convolutional layer
    def Model(self, k, c) :
        kernel_size = (3,3)
        pool_size = (2,2)
        pool_strides = (2,2)
        number_of_cells_limited = False

        inputs = tf.keras.Input(shape=self.input_shape)

        # convolutional base
        n = k
        multiplier = 2

        # ACFE block
        acfe = tf.keras.layers.Conv2D(filters=8, kernel_size=(1,9), strides=(1,pool_strides[0]), padding='valid',
                                      kernel_initializer=tf.keras.initializers.he_normal(), use_bias=False)(inputs)
        acfe = tf.keras.layers.BatchNormalization()(acfe)
        acfe = tf.keras.layers.ReLU()(acfe)

        acfe = tf.keras.layers.Conv2D(filters=64, kernel_size=(1,5), strides=(1,pool_strides[1]), padding='valid',
                                      kernel_initializer=tf.keras.initializers.he_normal(), use_bias=False)(acfe)
        acfe = tf.keras.layers.BatchNormalization()(acfe)
        acfe = tf.keras.layers.ReLU()(acfe)

        acfe = tf.keras.layers.MaxPooling2D(pool_size=(1,n))(acfe)

        x = tf.keras.layers.Permute((3, 2, 1))(acfe)

        # TDFE block
        # first convolutional layer
        x = tf.keras.layers.Conv2D(filters=32, kernel_size=kernel_size, strides=(1,1), padding='same',
                                      kernel_initializer=tf.keras.initializers.he_normal(), use_bias=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=pool_size)(x)

        # adding cells into TDFE
        for i in range(1, c + 1) :
            if x.shape[1] <= 1 or x.shape[2] <= 1 :
                number_of_cells_limited = True
                break;
            n = np.ceil(n * multiplier)
            multiplier = multiplier - 2**-i
            
            x = tf.keras.layers.Conv2D(filters=n, kernel_size=kernel_size, strides=(1,1), padding='same',
                                      kernel_initializer=tf.keras.initializers.he_normal(), use_bias=False)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.ReLU()(x)

            x = tf.keras.layers.MaxPooling2D(pool_size=pool_size)(x)
        
        # classifier
        x = tf.keras.layers.Dropout(rate=0.2)(x)

        x = tf.keras.layers.Conv2D(filters=self.num_classes, kernel_size=(1,1), strides=(1,1), padding='valid',
                                      kernel_initializer=tf.keras.initializers.he_normal(), use_bias=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)

        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(self.num_classes, kernel_initializer=tf.keras.initializers.he_normal())(x)
        outputs = tf.keras.layers.Softmax()(x)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)

        return model, number_of_cells_limited

    def plot_history(self, hist, model_name, model_save_dir, f=1):
        run_epochs = len(hist.history['accuracy'])
        tr_acc = hist.history['accuracy']
        tr_loss = hist.history['loss']
        val_acc = hist.history['val_acc']
        val_loss = hist.history['val_loss']

        save_epochs = [i for i in range(0, run_epochs, f)]
        save_tr_acc = [tr_acc[i]*100 for i in range(0, run_epochs, f)]
        save_tr_loss = [tr_loss[i] for i in range(0, run_epochs, f)]
        save_val_acc = [val_acc[i] for i in range(0, run_epochs, f)]
        save_val_loss = [val_loss[i] for i in range(0, run_epochs, f)]

        # Create a figure and axis
        fig, ax1 = plt.subplots()

        fig.set_figheight(12)
        fig.set_figwidth(24)

        # Plot accuracy lines
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy', color='black')
        ax1.plot(save_epochs, save_tr_acc, color='#800000', marker='o', label='Training Accuracy')
        ax1.plot(save_epochs, save_val_acc, color='#000075', marker='x', label='Validation Accuracy')
        # ax1.set_xticklabels(save_epochs, rotation=90)

        # Create a second y-axis for loss lines
        ax2 = ax1.twinx()  # Share the same x-axis
        ax2.set_ylabel('Loss', color='black')
        ax2.plot(save_epochs, save_tr_loss, color='#3cb44b', marker='s', label='Training Loss')
        ax2.plot(save_epochs, save_val_loss, color='#f58231', marker='^', label='Validation Loss')
        ax2.tick_params(axis='y', labelcolor='black')

        # Add a legend
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc='best')

        # Set a title
        plt.title('Accuracy and Loss Over Epochs')

        accuracy_matrices_path = model_save_dir

        curr_datetime = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
        filename = f'{model_name.lower()}-training_metrics_plot-{format(curr_datetime)}.png'

        if not os.path.exists(accuracy_matrices_path):
            os.makedirs(accuracy_matrices_path)

        # Save the plot to the specified folder
        destination = os.path.join(accuracy_matrices_path, filename)
        plt.savefig(destination, bbox_inches='tight')

    def evaluate_flash_and_peak_RAM_occupancy(self, custom_evaluator) :
        # quantize model to evaluate its peak RAM occupancy and its Flash occupancy
        self.quantize_model_uint8(custom_evaluator)

        # evaluate its peak RAM occupancy and its Flash occupancy using STMicroelectronics' X-CUBE-AI
        proc = subprocess.Popen(["./stm32tflm", f"{self.path_to_trained_models}/{self.model_name}_quantized.tflite"], stdout=subprocess.PIPE)
        try:
            outs, errs = proc.communicate(timeout=15)
            Flash, RAM = re.findall(r'\d+', str(outs))
        except subprocess.TimeoutExpired:
            proc.kill()
            outs, errs = proc.communicate()
            print("stm32tflm error")
            exit()

        return int(Flash), int(RAM)

    def quantize_model_uint8(self, custom_evaluator):
        def representative_dataset():
            rebatched_ds = tf.data.Dataset.from_tensor_slices((custom_evaluator.testX)).batch(1).take(100).prefetch(buffer_size=tf.data.AUTOTUNE)
            for data in rebatched_ds:
                yield [data]

        model = tf.keras.models.load_model(f"{self.path_to_trained_models}/{self.model_name}.h5")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8
        tflite_quant_model = converter.convert()

        with open(f"{self.path_to_trained_models}/{self.model_name}_quantized.tflite", 'wb') as f:
            f.write(tflite_quant_model)

        # os.remove(f"{self.path_to_trained_models}/{self.model_name}.h5")

    def evaluate_model_process(self, k, c) :
        if k > 0 :
            self.model_name = f"k_{k}_c_{c}"
            print(f"\n{self.model_name}\n")

            checkpoint = tf.keras.callbacks.ModelCheckpoint(
                f"{self.path_to_trained_models}/{self.model_name}.h5",
                monitor='val_acc', save_best_only=True, save_weights_only=False, mode='auto')

            model, number_of_cells_limited = self.Model(k, c)
            model.summary()

            loss = 'kullback_leibler_divergence'
            optimizer = keras.optimizers.SGD(learning_rate=self.trainer.opt.LR,
                                             weight_decay=self.trainer.opt.weightDecay,
                                             momentum=self.trainer.opt.momentum, nesterov=True)
            model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

            print(f'number_of_cells_limited: {number_of_cells_limited}')
            # exit()

            # learning schedule callback
            lrate = keras.callbacks.LearningRateScheduler(self.trainer.GetLR)
            custom_evaluator = CustomCallback(self.trainer.opt)

            callbacks_list = [lrate, custom_evaluator, checkpoint]

            # One epoch of training must be done before quantization, which is needed to evaluate RAM and Flash occupancy
            model.fit(self.trainer.trainGen, epochs=1,
                      steps_per_epoch=len(self.trainer.trainGen.data)//self.trainer.trainGen.batch_size,
                      callbacks=callbacks_list, verbose=0)

            model.save(f"{self.path_to_trained_models}/{self.model_name}.h5")

            Flash, RAM = self.evaluate_flash_and_peak_RAM_occupancy(custom_evaluator)
            print(f"\nRAM: {RAM},\t Flash: {Flash}\n")

            if Flash <= self.max_Flash and RAM <= self.max_RAM and not number_of_cells_limited :
                hist = model.fit(self.trainer.trainGen, epochs=self.trainer.opt.nEpochs,
                      steps_per_epoch=len(self.trainer.trainGen.data)//self.trainer.trainGen.batch_size,
                      callbacks=callbacks_list, verbose=0)

                self.plot_history(hist, self.model_name, self.path_to_trained_models, 5)
                self.quantize_model_uint8(custom_evaluator)

            return {'k': k,
                    'c': c if not number_of_cells_limited else "Not feasible",
                    'RAM': RAM if RAM <= self.max_RAM else "Outside the upper bound",
                    'Flash': Flash if Flash <= self.max_Flash else "Outside the upper bound",
                    'max_val_acc':
                    np.around(np.amax(hist.history['val_acc']), decimals=3)
                    if 'hist' in locals() else -3}
        else :
            return{'k': 'unfeasible', 'c': c, 'max_val_acc': -3}

    def explore_num_cells(self, k) :
        previous_architecture = {'k': -1, 'c': -1, 'max_val_acc': -2}
        current_architecture = {'k': -1, 'c': -1, 'max_val_acc': -1}
        c = 1
        k = int(k)

        while(current_architecture['max_val_acc'] > previous_architecture['max_val_acc']) :
            previous_architecture = current_architecture
            c = c + 1
            self.model_counter = self.model_counter + 1
            current_architecture = self.evaluate_model_process(k, c)
            print(f"\n\n\n{current_architecture}\n\n\n")
        return previous_architecture

    def search(self) :
        self.model_counter = 0
        epsilon = 0.005
        k0 = 8

        start = datetime.datetime.now()

        k = k0
        previous_architecture = self.explore_num_cells(k)
        k = 2 * k
        current_architecture = self.explore_num_cells(k)

        if (current_architecture['max_val_acc'] > previous_architecture['max_val_acc']) :
            previous_architecture = current_architecture
            k = 2 * k
            current_architecture = self.explore_num_cells(k)
            while(current_architecture['max_val_acc'] > previous_architecture['max_val_acc'] + epsilon) :
                previous_architecture = current_architecture
                k = 2 * k
                current_architecture = self.explore_num_cells(k)
        else :
            k = k0 / 2
            current_architecture = self.explore_num_cells(k)
            while(current_architecture['max_val_acc'] >= previous_architecture['max_val_acc']) :
                previous_architecture = current_architecture
                k = k / 2
                current_architecture = self.explore_num_cells(k)

        resulting_architecture = previous_architecture

        end = datetime.datetime.now()

        if (resulting_architecture['max_val_acc'] > 0) :
            resulting_architecture_name = f"k_{resulting_architecture['k']}_c_{resulting_architecture['c']}_quantized.tflite"
            self.path_to_resulting_architecture = f"{self.save_path}/resulting_architecture_{resulting_architecture_name}"
            print(f"\nResulting architecture: {resulting_architecture}\n")
        else :
            print(f"\nNo feasible architecture found\n")
        print(f"Elapsed time (search): {end-start}\n")

        return self.path_to_resulting_architecture

Enable stm32tflm script execution

In [ ]:
!chmod +x stm32tflm

Run ESC-NAS-Mixup

In [ ]:
opt = parse()
opt.sr = 20000
opt.inputLength = 30225
opt.split = 1
input_shape = (1, opt.inputLength, 1)

# 5 MB RAM, 150 kB Flash
peak_RAM_upper_bound = 1024 * 1024 * 5
Flash_upper_bound = 1024 * 150

val_split = 0.2

# save results
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
save_path = './results_US8K_' + timestamp

# show the GPU used
!nvidia-smi

ESCNAS_mixup = ESCNAS_mixup(peak_RAM_upper_bound, Flash_upper_bound,
                            val_split, input_shape, save_path=save_path, opt=opt, n_class=opt.nClasses)

In [ ]:
# search
path_to_tflite_model = ESCNAS_mixup.search()